In [1]:
import requests
import pandas as pd
import math
import pickle

In [2]:
funding_dict = pickle.load(open('funding_dict.pkl', 'rb'))
len(funding_dict)

179391

In [3]:
if funding_dict is None:
    funding_dict = {}
    for year in range(2016, 2024):
        print(f'loading {year} data')
        dump_url='https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/bso_dump'
        df = pd.read_csv(f'{dump_url}/bso-publications-latest_split_{year}_enriched.csv.gz', sep=';')

        for row in df.itertuples():
            anr, europe = [], []
            if isinstance(row.funding_anr, str):
                anr = row.funding_anr.split('|')
            if isinstance(row.funding_europe, str):
                europe = row.funding_europe.split('|')
            if anr or europe:
                if isinstance(row.doi, str):
                    doi = row.doi
                    #if doi not in funding_dict:
                    funding_dict[doi] = {'doi': doi, 'anr':anr, 'europe':europe}
    pickle.dump(funding_dict, open('funding_dict.pkl', 'wb'))                

In [12]:
grants_ia = {}
for year in range(2016, 2024):
    print(f'reading {year}')
    df = pd.read_json(f'publications_ai_fr_{year}.jsonl', lines=True, orient='records')
    for row in df.itertuples():
        if isinstance(row.doi, str):
            doi = row.doi.replace('https://doi.org/', '')
            if doi in funding_dict:
                for a in funding_dict[doi]['anr']:
                    if a not in grants_ia:
                        grants_ia[a] = {'id': a, 'type': 'ANR', 'publications':[]}
                    grants_ia[a]['publications'].append(doi)
                for a in funding_dict[doi]['europe']:
                    if a not in grants_ia:
                        grants_ia[a] = {'id': a, 'type': 'europe', 'publications':[]}
                    grants_ia[a]['publications'].append(doi)
                


reading 2016
reading 2017
reading 2018
reading 2019
reading 2020
reading 2021
reading 2022
reading 2023


In [15]:
df_grants_ia = pd.DataFrame(grants_ia.values())

In [16]:
df_grants_ia['nb_publications'] = df_grants_ia.publications.apply(lambda x:len(x))

In [18]:
df_grants_ia.to_json('grants_ia.jsonl', lines=True, orient='records')

In [19]:
df_grants_ia

,id,type,publications,nb_publications
0,ANR-10-IAIHU-06,ANR,"[10.1016/j.jalz.2016.02.002, 10.1073/pnas.1520...",102
1,ANR-10-INBS-09,ANR,"[10.1371/journal.pone.0157022, 10.1111/mec.138...",10
2,ANR-12-CORD-0016,ANR,[10.1109/tpami.2015.2487986],1
3,ANR-10-LABX-0087,ANR,"[10.1162/tacl_a_00115, 10.1016/j.neuron.2016.1...",65
4,640378,europe,[10.1126/science.aah3778],1
...,...,...,...,...
6493,ANR-10-COHO-05,ANR,[10.1002/hbm.26548],1
6494,ANR-16-LCV2-0006-01,ANR,[10.1002/hbm.26548],1
6495,ANR-10-IDEX-03-0,ANR,[10.1002/hbm.26548],1
6496,ANR-21-ESRE-0015),ANR,[10.1109/embc40787.2023.10340047],1
